#### Návrh RF filtru pro vstupní obvody přijímače rádiových signálů meteorů

Rádiové signály odražené od stop meteorů jsou frekvenčně modulovány pouze Dopplerovým posuvem proto mají šířku pásma pouze někoklik desítek kHz. Původní vysílaný signál je CW vysílání na frekvenci 143.05 kHz z pohledu RF techniky proto jde o filtr s velmi malou relativní šířkou pásma. 

Dalším cílem návrhu je, aby filtr byl poměrně jednoduše syntetizovatelný i pro jiné frekvence základního signálu tj. jiný vysílač. Nelze proto zvolit například krystalový nebo dutinový rezonátor, protože kusová výroba takových filtrů pro různé potřebné frekvence by byla velmi nákladná. Konstruční řešení proto musí být realizovatelné ze standardní řady L a C součástek. 

##### Zadání v podobě tolerančního pásma filtru 	
Značení parametrů stejné jako v programu Syntfil https://asinus.feld.cvut.cz/syntfil

Pásmová propust, Chebyshev, 3. řád
f-s = 142.551MHz, f-p = 143.025MHz, f_p = 143.075MHz, fs = 143.551MHz, fm = 143.05MHz, Δfp = 50kHz, Δfs = 1MHz, ap = 3dB, as = 60dB

##### Kanonická LC realizace
zakončení: symetrické, R = 50Ω, směr: zepředu, struktura: π

Výsledkem běžně používaných návrhových nástrojů je typicky následující struktura, která ale není realizovatelná z reálných součástek. 


![title](Filter_img/GRAVes_syntfil.png)

Takto navržená struktura je kanonickou strukturou, která začíná být prakticky realizovatelná od situace kdy šířka pásma filtru je alespoň polovina pracovní frekvence. Ale i tak jsou potřebné hodnoty součástek těžko zkontruovatelné, protože nutné rozměry vytvářejí potíže s vlastní rezonancí. Existuje ještě několik analyticky správných, ale technicky těžko realizovatelných struktur jako eliptické filtry, Nortonovy transformované filtry atd.

Pro náš případ pásmové propusti lze však využít požadavku na velmi malou šířku pásma, která umožňuje uvažovat o aproximacích platných pro danou frekvenci. Pro změnu impedance mezi jednotlivými bloky filtru tak lze například použít LC články místo trasformátorů z vázaných indukčostí. 

##### Přímo vázaná LC struktura

Filtry tohoto typu se obecně skládají z přizpůsobovacích bloků, rezonátorů. Tato obvodová realizace filtru může využívat obrovské množství různých topologií rezonátorů a přizpůsobovacích obvodů, které lze vzájemně kombinovat. Na následujícím schématu je jedna ze základních struktur.

![Direct coupled Filter](Filter_img/Direct_coupled_RFfilter.png)

Iterativně 


In [15]:
#  https://github.com/ahkab/ahkab
from ahkab import new_ac, run
from ahkab.circuit import Circuit
from ahkab.plotting import plot_results # calls matplotlib for you
import numpy as np

# Define the circuit
cir = Circuit('Butterworth 1kHz band-pass filter')
cir.add_vsource('V1', 'n1', cir.gnd, dc_value=0., ac_value=1.)
cir.add_resistor('R1', 'n1', 'n2', 50.)
cir.add_inductor('L1', 'n2', 'n3', 0.245894)
cir.add_capacitor('C1', 'n3', 'n4', 1.03013e-07)
cir.add_inductor('L2', 'n4', cir.gnd, 9.83652e-05)
cir.add_capacitor('C2', 'n4', cir.gnd, 0.000257513)
cir.add_inductor('L3', 'n4', 'n5', 0.795775)
cir.add_capacitor('C3', 'n5', 'n6', 3.1831e-08)
cir.add_inductor('L4', 'n6', cir.gnd, 9.83652e-05)
cir.add_capacitor('C4', 'n6', cir.gnd, 0.000257513)
cir.add_capacitor('C5', 'n7', 'n8', 1.03013e-07)
cir.add_inductor('L5', 'n6', 'n7', 0.245894)
cir.add_resistor('R2', 'n8', cir.gnd, 50.)

# Define the analysis
ac1 = new_ac(.97e3, 1.03e3, 1e2, x0=None)

# run it
res = run(cir, ac1)

# plot the results
plot_results('5th order 1kHz Butterworth filter', [('|Vn8|',"")], res['ac'])

In [16]:
print cir

* Butterworth 1kHz band-pass filter
V1 n1 0 type=vdc value=0.0 vac=1.0 
R1 n1 n2 50
L1 n2 n3 0.245894
C1 n3 n4 1.03013e-07
L2 n4 0 9.83652e-05
C2 n4 0 0.000257513
L3 n4 n5 0.795775
C3 n5 n6 3.1831e-08
L4 n6 0 9.83652e-05
C4 n6 0 0.000257513
C5 n7 n8 1.03013e-07
L5 n6 n7 0.245894
R2 n8 0 50


In [1]:
from ahkab.plotting import show_plots
show_plots()

ImportError: No module named ahkab.plotting

![Předpokládaná výsledná realizace filtru](Filter_img/BP01B_schematics.png)


#### Reference 

http://www.iowahills.com/9RFFiltersPage.html 

